In [2]:
# from google.colab import drive

# drive.mount('/content/drive')

In [3]:
# !unzip drive/MyDrive/edsa-movie-recommendation-2022.zip > /dev/null

In [4]:
from matplotlib import figure
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from multiprocesspandas import applyparallel
plt.rcParams['figure.figsize'] = (15, 10)

In [5]:
def convert_columns(data):
  
  for col in data.columns:
    if data[col].dtype == 'object':
      data[col] = data[col].astype('category')
    
    elif data[col].dtype == 'int64':
      data[col] = data[col].astype('int32')
    
    else:
      data[col] = data[col].astype('float32')
    
  return data

In [6]:
imdb_data = pd.read_csv('imdb_data.csv')
train = pd.read_csv('train.csv')
train = convert_columns(train)
test = pd.read_csv('test.csv')
test = convert_columns(test)
movies = pd.read_csv('movies.csv')
# movies = convert_columns(movies)
sample_submission = pd.read_csv('sample_submission.csv')

In [7]:
def aggregate(series):
  aggregate = []
  for i in series:
    for j in i:
      aggregate.append(j)
  
  return aggregate

In [8]:
def split_text(text, sep):
  split_text = text.split(sep)
  return split_text

def join_lists(list_):
  return ' '.join(list_)

In [9]:
movies['genres'] = movies['genres'].apply(split_text, args=('|',))

In [10]:
movies['year'] = movies['title'].apply(lambda x: x[-7:].replace('(', '').replace(')', ''))
movies['title'] = movies['title'].apply((lambda x: x[:-7]))

In [11]:
imdb_data_copy = imdb_data.copy(deep=True)
imdb_data_copy.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [12]:
imdb_data_copy = imdb_data_copy.drop('budget', axis=1)
imdb_data_copy = imdb_data_copy.dropna()

In [13]:
imdb_data_copy['title_cast'] = imdb_data_copy['title_cast'].apply(split_text, args=("|", ))

In [14]:
imdb_data_copy['plot_keywords'] = imdb_data_copy['plot_keywords'].apply(split_text, args=("|", )).apply(join_lists).apply(split_text, args=(" ", ))

In [15]:
# actors= aggregate(imdb_data_copy.title_cast)
# print(f'there are {len(set(actors))} different actors')

In [16]:
# pk= aggregate(imdb_data_copy.plot_keywords)
# print(f'there are {len(set(pk))} different plot keywords')

In [17]:
# director= aggregate(imdb_data_copy.director)
# print(f'there are {len(set(director))} different directors')

In [18]:
imdb_movies = imdb_data_copy.merge(movies, on='movieId')

In [19]:
del imdb_data_copy
del imdb_data

In [20]:
imdb_movies.shape

(12632, 8)

In [21]:
imdb_movies.head()

,movieId,title_cast,director,runtime,plot_keywords,title,genres,year
0,1,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,81.0,"[toy, rivalry, cowboy, cgi, animation]",Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Jonathan Hensleigh,104.0,"[board, game, adventurer, fight, game]",Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,"[Walter Matthau, Jack Lemmon, Sophia Loren, An...",Mark Steven Johnson,101.0,"[boat, lake, neighbor, rivalry]",Grumpier Old Men,"[Comedy, Romance]",1995
3,4,"[Whitney Houston, Angela Bassett, Loretta Devi...",Terry McMillan,124.0,"[black, american, husband, wife, relationship,...",Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Albert Hackett,106.0,"[fatherhood, doberman, dog, mansion]",Father of the Bride Part II,[Comedy],1995


In [22]:
cols = ['movieId', 'title_cast', 'movie_director', 'runtime', 'plot_keywords', 'title', 'genres', 'year']

imdb_movies.columns = cols

In [23]:
from sklearn.feature_extraction.text import CountVectorizer


In [24]:
cv = CountVectorizer()
count_vector=cv.fit_transform(' '.join(x) for x in imdb_movies.genres)
count_vect_df = pd.DataFrame(count_vector.todense(), columns = cv.get_feature_names_out())
count_vect_df.shape

(12632, 24)

In [25]:
imdb_movies = pd.concat([imdb_movies,count_vect_df], axis=1)

In [26]:
# plot = CountVectorizer()
# plot_vector= plot.fit_transform(' '.join(x) for x in imdb_movies.plot_keywords)
# plot_vect_df = pd.DataFrame(plot_vector.todense(), columns = plot.get_feature_names_out())

# plot_vect_df.shape

In [27]:
# imdb_movies = pd.concat([imdb_movies,plot_vect_df], axis=1)

### segregate stars

In [28]:
imdb_movies.drop(['genres', 'plot_keywords'], axis=1, inplace=True)

In [29]:
def feature_importance(series):
  items = aggregate(series)
  frequency = {}

  for item in items:
    if item in frequency:
      frequency[item] += 1
    else:
      frequency[item] = 1
    
  sorted_freq = {k: v for k, v in sorted(frequency.items(), reverse=True, key=lambda item: item[1])}

  return sorted_freq

In [30]:
# engineer a_list_actors, b_list_actors, c_list_actors

def classify(df, col, columns=[], threshold=[]):

  most_featured = feature_importance(df[col])
  most_featured_df = pd.DataFrame.from_dict(most_featured.items())
  most_featured_df.columns = columns

  print('feature extraction complete')

  a_list_actors = most_featured_df[columns[0]][most_featured_df[columns[1]] >= threshold[0]].tolist()
  b_list_actors = most_featured_df[columns[0]][(most_featured_df[columns[1]] >= threshold[1]) & (most_featured_df[columns[1]] < threshold[0])].tolist()
  c_list_actors = most_featured_df[columns[0]][most_featured_df[columns[1]] < threshold[1]].tolist()

  a_list_count = []
  b_list_count = []
  c_list_count = []

  for row in df[col]:
    a_counter = 0
    b_counter = 0
    c_counter = 0

    for item in row:
      if item in a_list_actors:
        a_counter += 1
      if item in b_list_actors:
        b_counter += 1
      if item in c_list_actors:
        c_counter += 1

    a_list_count.append(a_counter)
    b_list_count.append(b_counter)
    c_list_count.append(c_counter)
  
  return (a_list_count, b_list_count, c_list_count)

In [31]:
imdb_movies['movie_director'] = imdb_movies['movie_director'].apply(lambda x: x.split('|') )

In [32]:
a, b, c = classify(imdb_movies, 'title_cast', columns=['actor', 'appearance'], threshold=[30, 15])

feature extraction complete


In [33]:
d, e, f = classify(imdb_movies, 'movie_director', columns=['dir', 'jobs'], threshold=[5, 2])

feature extraction complete


In [34]:
imdb_movies['a_actors'], imdb_movies['b_actors'], imdb_movies['c_actors'] = a, b, c
imdb_movies['a_directors'], imdb_movies['b_directors'], imdb_movies['c_directors'] = d, e, f

In [35]:
del a
del b
del c
del d
del e
del f

In [36]:
imdb_movies.head()

,movieId,title_cast,movie_director,runtime,title,year,action,adventure,animation,children,...,sci,thriller,war,western,a_actors,b_actors,c_actors,a_directors,b_directors,c_directors
0,1,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],81.0,Toy Story,1995,0,1,1,1,...,0,0,0,0,2,2,11,0,1,0
1,2,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Jonathan Hensleigh],104.0,Jumanji,1995,0,1,0,1,...,0,0,0,0,3,3,9,1,0,0
2,3,"[Walter Matthau, Jack Lemmon, Sophia Loren, An...",[Mark Steven Johnson],101.0,Grumpier Old Men,1995,0,0,0,0,...,0,0,0,0,1,1,13,1,0,0
3,4,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Terry McMillan],124.0,Waiting to Exhale,1995,0,0,0,0,...,0,0,0,0,0,5,10,0,1,0
4,5,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Albert Hackett],106.0,Father of the Bride Part II,1995,0,0,0,0,...,0,0,0,0,0,6,9,0,0,1


In [37]:
drop_cols = ['title_cast', 'movie_director', 'title']

imdb_movies.drop(drop_cols, axis=1, inplace=True)

In [38]:
imdb_movies.year = imdb_movies.year.apply(lambda x: x.strip())
imdb_movies.year = imdb_movies.year.apply(lambda x: int(x) if x.isnumeric() else np.NaN)

In [39]:
imdb_movies = convert_columns(imdb_movies)

In [40]:
def stringify(num):
  num = str(num)
  return num

In [41]:
train['userId'] = train['userId'].apply(stringify)
train['movieId'] = train['movieId'].apply(stringify)

In [42]:
train['Id'] = train.apply(lambda x: x.userId + '_' + x.movieId, axis=1)

In [43]:
train_drop = ['timestamp', 'userId']

train.drop(train_drop, axis=1, inplace=True)

In [44]:
train.head()

,movieId,rating,Id
0,57669,4.0,5163_57669
1,5,4.5,106343_5
2,5459,5.0,146790_5459
3,32296,2.0,106362_32296
4,366,3.0,9041_366


## Merge IMDB Copy and Train

In [45]:
# # making merger
train['movieId'] = train['movieId'].astype('int32')
full_data = train.merge(imdb_movies, how='left', left_on='movieId', right_on='movieId')

In [46]:
full_data.head()

,movieId,rating,Id,runtime,year,action,adventure,animation,children,comedy,...,sci,thriller,war,western,a_actors,b_actors,c_actors,a_directors,b_directors,c_directors
0,57669,4.0,5163_57669,107.0,2008.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,2.0,1.0,12.0,0.0,1.0,0.0
1,5,4.5,106343_5,106.0,1995.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,6.0,9.0,0.0,0.0,1.0
2,5459,5.0,146790_5459,88.0,2002.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,2.0,8.0,5.0,0.0,1.0,0.0
3,32296,2.0,106362_32296,115.0,2005.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,5.0,9.0,1.0,0.0,0.0
4,366,3.0,9041_366,112.0,1994.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,14.0,1.0,0.0,0.0


In [47]:
full_data = full_data.drop('movieId', axis=1)

In [48]:
full_data.shape

(10000038, 33)

In [49]:
del train
del imdb_movies

In [50]:
cols = list(full_data.isnull().sum()[full_data.isnull().sum()>1].index)

In [51]:
def fill_na(df, cols=[]):
  for col in cols:
    df[col] = df[col].fillna(df[col].median())
  
  return df

In [52]:
full_data = fill_na(full_data, cols=cols)

In [53]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor 

In [54]:
X = full_data.drop(['rating', 'Id'], axis=1)
y = full_data['rating']

In [55]:
scaler = StandardScaler()

In [1]:
X_scaled = scaler.fit_transform(X)

NameError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=101)

In [ ]:
reg_lgbm = LGBMRegressor(learning_rate=0.01, max_depth=10, num_leaves=3)
# rf = RandomForestRegressor()
xgb = XGBRegressor()

In [ ]:
reg_lgbm.fit(X_train, y_train)
# rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

In [ ]:
y_pred = reg_lgbm.predict(X_test)
# y_rf = rf.predict(X_test)
y_xgb = xgb.predict(X_test)

In [ ]:
# MSE =sum((y_test - y_pred)**2)/y_test.shape[0]

In [ ]:
# RMSE = np.sqrt(MSE)

In [ ]:
# RMSE

1.0516486765733954

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred))
# rmse_rf = np.sqrt(mean_squared_error(y_test, y_rf))
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_xgb))

In [ ]:
print(
    f'LGBM : {rmse_lgbm}\n
    XGB: {rmse_xgb}'
)

1.0516486765731183

In [ ]:
mse

1.1059649389379913

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'learning_rate':[0.01, 0.001], 'boosting':['goss', 'dart'], 'max_depth': [10, 15], 'num_leaves':[2,3]}

In [ ]:
grid = GridSearchCV(reg_lgbm, params, cv=5, verbose=3)

In [ ]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=2;, score=0.014 total time=  23.1s
[CV 2/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=2;, score=0.014 total time=  20.1s
[CV 3/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=2;, score=0.014 total time=  19.8s
[CV 4/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=2;, score=0.014 total time=  20.3s
[CV 5/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=2;, score=0.014 total time=  19.9s
[CV 1/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=3;, score=0.017 total time=  28.6s
[CV 2/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=3;, score=0.017 total time=  27.5s
[CV 3/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=3;, score=0.017 total time=  28.4s
[CV 4/5] END boosting=goss, learning_rate=0.01, max_depth=10, num_leaves=3;

GridSearchCV(cv=5,
             estimator=LGBMRegressor(boosting='dart', learning_rate=0.001,
                                     max_depth=15, num_leaves=2),
             param_grid={'boosting': ['goss', 'dart'],
                         'learning_rate': [0.01, 0.001], 'max_depth': [10, 15],
                         'num_leaves': [2, 3]},
             verbose=3)

In [ ]:
grid.best_params_

{'boosting': 'goss', 'learning_rate': 0.01, 'max_depth': 10, 'num_leaves': 3}

In [ ]:
grid.best_estimator_

LGBMRegressor(boosting='goss', learning_rate=0.01, max_depth=10, num_leaves=3)